In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# Import the new model
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import joblib
joblib.dump(model, 'gold_price_model.pkl')
loaded_model = joblib.load('gold_price_model.pkl')
#result = loaded_model.predict(new_data)



# 1. Load and Clean Data (Same as before)
df = pd.read_csv('/content/gld_price_data.csv')
x = df.drop(['Date', 'GLD'], axis=1)
y = df['GLD']

# 2. Split Data (Same as before for consistency)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 3. Model Training (NEW MODEL)
# Initialize the Random Forest Regressor
# n_estimators=100 is a common starting point (100 decision trees)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

# 4. Predictions and Evaluation (Same as before)
predictions = model.predict(x_test)

mae = metrics.mean_absolute_error(y_test, predictions)
mse = metrics.mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2_score = metrics.r2_score(y_test, predictions)

# 5. Output Results
print("--- Random Forest Model Summary ---")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared Score: {r2_score:.4f}")


: 